# Trabajo Práctico N°1: Fundamentos del Qubit
**Introducción a la Computación Cuántica — CACIC 2025**

Este notebook desarrolla las consignas del día 1, basadas en los temas:
- Bases alternativas (X, Y, Z)
- Notación de Dirac y productos internos
- Matrices de Pauli
- Estados de espín en direcciones arbitrarias

Cada ejercicio se resuelve analíticamente y se verifica mediante cálculo simbólico y numérico en Python usando **NumPy** y **Qiskit**.

In [2]:
import numpy as np
from numpy import sqrt
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_bloch_vector
import matplotlib.pyplot as plt
%matplotlib inline


## 1. Bases alternativas

### 1.1 Base X
Definimos:
$$
|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle), \quad
|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)
$$
Verificaremos ortonormalidad y completitud.

In [3]:
# Vectores columna base Z
ket0 = np.array([[1],[0]])
ket1 = np.array([[0],[1]])

# Estados de la base X
ket_plus = (ket0 + ket1)/sqrt(2)
ket_minus = (ket0 - ket1)/sqrt(2)

# Productos internos
print("⟨+|+⟩ =", np.vdot(ket_plus, ket_plus))
print("⟨-|−⟩ =", np.vdot(ket_minus, ket_minus))
print("⟨+|−⟩ =", np.vdot(ket_plus, ket_minus))

# Matrices de proyección
proj_plus = ket_plus @ ket_plus.T.conj()
proj_minus = ket_minus @ ket_minus.T.conj()

# Completitud
I = proj_plus + proj_minus
print("\n|+><+| + |-><-| =\n", I)


⟨+|+⟩ = 0.9999999999999998
⟨-|−⟩ = 0.9999999999999998
⟨+|−⟩ = 0.0

|+><+| + |-><-| =
 [[1. 0.]
 [0. 1.]]


### 1.2 Base Y
$$
|+i\rangle = \frac{1}{\sqrt{2}}(|0\rangle + i|1\rangle), \quad
|-i\rangle = \frac{1}{\sqrt{2}}(|0\rangle - i|1\rangle)
$$

Verificamos ortonormalidad y expresamos |+⟩ en esta base.


In [4]:
# Base Y
ket_plus_i = (ket0 + 1j*ket1)/sqrt(2)
ket_minus_i = (ket0 - 1j*ket1)/sqrt(2)

# Ortonormalidad
print("⟨+i|+i⟩ =", np.vdot(ket_plus_i, ket_plus_i))
print("⟨-i|-i⟩ =", np.vdot(ket_minus_i, ket_minus_i))
print("⟨+i|-i⟩ =", np.vdot(ket_plus_i, ket_minus_i))

# Expresión de |+> en base Y
alpha = np.vdot(ket_plus_i, ket_plus)
beta = np.vdot(ket_minus_i, ket_plus)
print("\nα =", alpha, " β =", beta)
print("Verificación: α|+i> + β|-i> =", alpha*ket_plus_i + beta*ket_minus_i)


⟨+i|+i⟩ = (0.9999999999999998+0j)
⟨-i|-i⟩ = (0.9999999999999998+0j)
⟨+i|-i⟩ = 0j

α = (0.4999999999999999-0.4999999999999999j)  β = (0.4999999999999999+0.4999999999999999j)
Verificación: α|+i> + β|-i> = [[0.70710678+0.j]
 [0.70710678+0.j]]


## 2. Matrices de Pauli

Usamos la notación:
$$
X = |+\rangle⟨+| - |−⟩⟨−|, \quad
Y = |+i\rangle⟨+i| - |−i\rangle⟨−i|, \quad
Z = |0\rangle⟨0| - |1\rangle⟨1|
$$
y verificamos sus propiedades.


In [5]:
# Construcción explícita de las matrices de Pauli
X = proj_plus - proj_minus
Y = (ket_plus_i @ ket_plus_i.T.conj()) - (ket_minus_i @ ket_minus_i.T.conj())
Z = (ket0 @ ket0.T.conj()) - (ket1 @ ket1.T.conj())

print("X =\n", X)
print("Y =\n", Y)
print("Z =\n", Z)

# Propiedades
for name, M in zip(["X","Y","Z"], [X,Y,Z]):
    print(f"\n{name}² =\n", M@M)
    print(f"{name}† =\n", M.T.conj())
    print("Hermiticity:", np.allclose(M, M.T.conj()))
    print("Trace =", np.trace(M))


X =
 [[0. 1.]
 [1. 0.]]
Y =
 [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
Z =
 [[ 1  0]
 [ 0 -1]]

X² =
 [[1. 0.]
 [0. 1.]]
X† =
 [[0. 1.]
 [1. 0.]]
Hermiticity: True
Trace = 0.0

Y² =
 [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Y† =
 [[0.-0.j 0.-1.j]
 [0.+1.j 0.-0.j]]
Hermiticity: True
Trace = 0j

Z² =
 [[1 0]
 [0 1]]
Z† =
 [[ 1  0]
 [ 0 -1]]
Hermiticity: True
Trace = 0


### Anticonmutación
Verificamos que las matrices cumplen:
$$
XY + YX = 0
$$


In [6]:
XY_plus_YX = X@Y + Y@X
print("XY + YX =\n", XY_plus_YX)


XY + YX =
 [[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


## 3. Estados en dirección arbitraria

$$
|\psi(θ,ϕ)\rangle = \cos\frac{θ}{2}|0\rangle + e^{iϕ}\sin\frac{θ}{2}|1\rangle
$$

Verificamos que para diferentes ángulos obtenemos las bases X, Y y Z.


In [7]:
def state(theta, phi):
    return np.array([[np.cos(theta/2)], [np.exp(1j*phi)*np.sin(theta/2)]])

# Casos específicos
cases = [
    ("+x̂", np.pi/2, 0),
    ("+ŷ", np.pi/2, np.pi/2),
    ("+ẑ", 0, 0)
]

for name, th, ph in cases:
    print(f"{name}:")
    print(state(th, ph))
    print()


+x̂:
[[0.70710678+0.j]
 [0.70710678+0.j]]

+ŷ:
[[7.07106781e-01+0.j        ]
 [4.32978028e-17+0.70710678j]]

+ẑ:
[[1.+0.j]
 [0.+0.j]]



### Verificación como autovector

La matriz de Pauli en dirección arbitraria:
$$
σ_n = \sinθ\cosϕ X + \sinθ\sinϕ Y + \cosθ Z
$$
Debe cumplir:
$$
σ_n |ψ(θ,ϕ)\rangle = +|ψ(θ,ϕ)\rangle
$$

In [8]:
def sigma_n(theta, phi):
    return np.sin(theta)*np.cos(phi)*X + np.sin(theta)*np.sin(phi)*Y + np.cos(theta)*Z

# Verificamos para dirección +x̂
theta, phi = np.pi/2, 0
psi = state(theta, phi)
sigma = sigma_n(theta, phi)
res = sigma @ psi
print("σ_n |ψ> =\n", res)
print("¿Autovector con +1?", np.allclose(res, psi))


σ_n |ψ> =
 [[0.70710678+0.j]
 [0.70710678+0.j]]
¿Autovector con +1? True


## Conclusiones

- Se verificó que las bases X, Y y Z son ortonormales y completas.  
- Las matrices de Pauli son hermíticas, unitarias y de traza nula.  
- Los estados en direcciones arbitrarias pueden representarse mediante parámetros (θ, ϕ) y son autovectores de σₙ con autovalor +1.  
- Todo el formalismo concuerda con las propiedades teóricas del qubit y se puede extender a representaciones en la esfera de Bloch.
